[View in Colaboratory](https://colab.research.google.com/github/aksh98/Attribute_Classification/blob/master/BTP_Apr_16.ipynb)

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torch
!pip3 install torchvision


    94% |██████████████████████████████▍ | 561.4MB 17.2MB/s eta 0:00:02

    100% |████████████████████████████████| 592.3MB 65.7MB/s 
    100% |████████████████████████████████| 51kB 1.7MB/s 
    100% |████████████████████████████████| 2.0MB 645kB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision


In [4]:
# ========================================================

# State of the art Accuracy - 91.25

# ========================================================


import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.io
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import Normalize
import os
import pickle
from torchvision import datasets, transforms
import torchvision.utils as vutil
import argparse
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from argparse import ArgumentParser
from sklearn.model_selection import train_test_split
print("Done")


Done


In [5]:

%cd drive
%cd Colab Notebooks 
#/content/drive/Colab Notebooks
!ls

/content/drive
/content/drive/Colab Notebooks
BTP_10thApr.ipynb	      btpfinalattributes.csv  output1.txt
BTP1.ipynb		      BTP.ipynb		      output21.txt
btpaligned2		      finallala.txt	      output22.txt
btpaligned2.zip		      full_attributes.txt     output2.txt
btpaligned.zip		      imgarr_3513.pkl	      output.txt
BTP_Apr_16.ipynb	      imgarr_3788.pkl	      tiny_attributes.txt
BTPattributefile.csv	      IP.ipynb		      tiny_dataset
BTP_Attributes.ods	      lfw_aligned	      Untitled0.ipynb
btpattributesrefined.csv      lfwcolorfaces.zip       Untitled1.ipynb
btpattributesrefined.csv.ods  lfw_crop_A	      Untitled2.ipynb
btpdataset.txt		      __MACOSX


In [106]:
# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
mini_batch_size = 64

transform = transforms.Compose([
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) ])

      
def readcsv(filename):
    ifile = open(filename, "rU")
    reader = csv.reader(ifile, delimiter=",")
    rownum = 0
    a = []

    for row in reader:
        a.append (row)
        rownum += 1
    
    ifile.close()
    return a


# ========================================================

# Load Attributes

# ========================================================


x = np.loadtxt('tiny_attributes.txt',delimiter = '\n',dtype=np.str)
print(len(x))
array = []
# h = 3788
w = 41
h = 8

# h, w = 13143, 41 

matrix = [[0 for x in range(w)] for y in range(h-1)] 
images = []

# Extracting/creating image name from attribute file

for i in range(1,h):
    arr = x[i].split() 
    name_len = len(arr)-41  # 41 = attributes considered here

    la = arr[0]+"_"
    for l in range(1,name_len-1):
        la += arr[l]+"_"
    if(float(arr[name_len-1]) > 99):
        la += "0" + arr[name_len-1]+".jpg"
    elif(float(arr[name_len-1]) > 9 and float(arr[name_len]) < 100):
        la += "00" + arr[name_len-1] +".jpg"    
    elif(float(arr[name_len-1]) < 10):
        la += "000" + arr[name_len-1] +".jpg"
    print(la)
    images.append(la)
    
    for j in range(name_len-1,len(arr)-1):
        matrix[i-1][j-name_len+1] = float(arr[j+1])

print("Matrix size  = ",len(matrix),len(matrix[0]))
print("images len",len(images))
matrix = np.array(matrix)
matrix = matrix.transpose()

mini = np.amin(matrix,axis = 1)
maxi = np.amax(matrix,axis = 1)
for i in range(0,w): # = 41
    for j in range(0,h-1):
        matrix[i][j] = float((matrix[i][j]-mini[i])/float(maxi[i]-mini[i]))

matrix = matrix.transpose()


8
Aaron_Eckhart_0001.jpg
Aaron_Guiel_0001.jpg
Aaron_Patterson_0001.jpg
Aaron_Peirsol_0001.jpg
Aaron_Peirsol_0002.jpg
Aaron_Peirsol_0003.jpg
Aaron_Peirsol_0004.jpg
Matrix size  =  7 41
images len 7


In [107]:

# ========================================================

# Load images into img_arr

# ========================================================

img_arr = []
i = 0
img_size = 0
for i in range(0,h-1): 
    img = images[i]
    imgpath = os.path.join("lfw_aligned/",img)
    image = cv2.imread(imgpath)    
    if(image is None):
        print("No Image loaded")
        print("Useless Image => ",img)
    else:
        image = np.reshape(image,(image.shape[0]*image.shape[1])*image.shape[2])
        img_size = image.shape[0]
        image = [x / 255 for x in image]
        img_arr.append(image)
    mat = matrix[i][:]
#     print(mat)
    if(i%100 == 0):
        print(i,end=" ")
print("Img_Arr length = ",len(img_arr))

# img_arr(1,62500) and matrix(13143,74)


0 Img_Arr length =  7


In [0]:
# ========================================================

# Load images from pickle file or dump them into pkl.. Save time :) 

# ========================================================

# print(len(img_arr), len(img_arr[0]))
# print(img_arr[2280][:])
# print(mydict)
f = open('imgarr_3788.pkl', 'rb')
mydict = pickle.load(f)
# mydict = pickle.load("imgarr_3788.pkl")
print(len(mydict))
# pickle.dump(img_arr,open("imgarr_3788.pkl","wb"))

In [108]:
import sys
X_train, X_test, y_train, y_test = train_test_split( img_arr, matrix, test_size=0.20, random_state=24)
print(len(X_train),len(X_train[0]))
print(len(y_train),len(y_train[0]))

5 187500
5 41


In [151]:

# ========================================================

# Architecture 

# ========================================================

img_size = 250
import operator
import numpy as np
 
def mask_size_helper(args):
    return reduce(operator.mul, args) 
 
def create_dropconnect_mask(dc_keep_prob, dimensions):
    mask_vector = np.random.binomial(1, dc_keep_prob, mask_size_helper(dimensions))
    mask_array = mask_vector.reshape(dimensions)
    return mask_array
 
def dropconnect(W, dc_keep_prob):
    dimensions = W.shape
    return W * create_dropconnect_mask(dc_keep_prob, dimensions)
  
# def dropconnect(Wt, prob):
#     return F.dropout(Wt, p=prob) * prob
  
class InceptionB(nn.Module):
    def __init__(self,in_channels):
        super(InceptionB,self).__init__()
        # Around Hair
        self.fco_11 = nn.Linear(in_channels,512)
        self.fco_12 = nn.Linear(512,512)
        self.fco_13 = nn.Linear(512,13)

        # Facial Hair
        self.fco_21 = nn.Linear(in_channels,512)
        self.fco_22 = nn.Linear(512,512)
        self.fco_23 = nn.Linear(512,5)

        # Cheeks
        self.fco_31 = nn.Linear(in_channels,512)
        self.fco_32 = nn.Linear(512,512)
        self.fco_33 = nn.Linear(512,2)

        # Fat
        self.fco_41 = nn.Linear(in_channels,512)
        self.fco_42 = nn.Linear(512,512)
        self.fco_43 = nn.Linear(512,2)

        
    def forward(self,x):
        br1 = F.relu(self.fco_11(x))
#         br1 = F.dropout(br1,p=0.5)
        br1 = dropconnect(br1,0.5)
        br1 = F.relu(self.fco_12(br1))
        br1 = dropconnect(br1,0.5)
        br1 = self.fco_13(br1)
        
        br2 = F.relu(self.fco_21(x))
        br2 = dropconnect(br2,0.5)
        br2 = F.relu(self.fco_22(br2))
        br2 = dropconnect(br2,0.5)
        br2 = self.fco_23(br2)
        
        br3 = F.relu(self.fco_31(x))
        br3 = dropconnect(br3,0.5)
        br3 = F.relu(self.fco_32(br3))
        br3 = dropconnect(br3,0.5)
        br3 = self.fco_33(br3)
        
        br4 = F.relu(self.fco_41(x))
        br4 = dropconnect(br4,0.5)
        br4 = F.relu(self.fco_42(br4))
        br4 = dropconnect(br4,0.5)
        br4 = self.fco_43(br4)
        
        output = [br1,br2,br3,br4]

        return torch.cat(output,1)

# ==============================================================
# ==============================================================


class InceptionA(nn.Module):
    
    def __init__(self,in_channels):  
        super(InceptionA, self).__init__()
        channel = 4800
        # Gender
        self.conv3g = nn.Conv2d(in_channels, 300, kernel_size = 3)
        # relu - 5*5 pooling - 5*5 normalization
        self.fc1g = nn.Linear(channel,512)
        self.fc2g = nn.Linear(512,512)
        self.fc3g = nn.Linear(512,1)
        
        # Eyes 
        self.conv3e = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1e = nn.Linear(channel,512)
        self.fc2e = nn.Linear(512,512)
        self.fc3e = nn.Linear(512,5)

        # Nose
        self.conv3n = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1n = nn.Linear(channel,512)
        self.fc2n = nn.Linear(512,512)
        self.fc3n = nn.Linear(512,2)

        # Face
        self.conv3f = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1f = nn.Linear(channel,512)
        self.fc2f = nn.Linear(512,512)
        self.fc3f = nn.Linear(512,6)

        # Mouth 
        self.conv3m = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1m = nn.Linear(channel,512)
        self.fc2m = nn.Linear(512,512)
        self.fc3m = nn.Linear(512,4)

        # Others
        self.conv3o = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.others = InceptionB(channel)
        
        
    def forward(self,x):
        b1 = F.relu(self.conv3g(x))
        b1 = F.max_pool2d(b1, kernel_size=5)
        b1 = b1.view(1,4800)
        b1 = F.relu(self.fc1g(b1))
#         b1 = F.dropout(b1,p=0.5)
        b1 = dropconnect(b1,0.5)
        b1 = F.relu(self.fc2g(b1))
        b1 = dropconnect(b1,0.5)
        b1 = self.fc3g(b1)
        
        b2 = F.relu(self.conv3e(x))
        b2 = F.max_pool2d(b2, kernel_size=5)
        b2 = b2.view(1,4800)
        b2 = F.relu(self.fc1e(b2))
        b2 = dropconnect(b2,0.5)
        b2 = F.relu(self.fc2e(b2))
        b2 = dropconnect(b2,0.5)
        b2 = self.fc3e(b2)
        
        b3 = F.relu(self.conv3n(x))
        b3 = F.max_pool2d(b3, kernel_size=5)
        b3 = b3.view(1,4800)
        b3 = F.relu(self.fc1n(b3))
        b3 = dropconnect(b3,0.5)
        b3 = F.relu(self.fc2n(b3))
        b3 = dropconnect(b3,0.5)
        b3 = self.fc3n(b3)
        
        b4 = F.relu(self.conv3f(x))
        b4 = F.max_pool2d(b4, kernel_size=5)
        b4 = b4.view(1,4800)
        b4 = F.relu(self.fc1f(b4))
        b4 = dropconnect(b4,0.5)
        b4 = F.relu(self.fc2f(b4))
        b4 = dropconnect(b4,0.5)
        b4 = self.fc3f(b4)
        
        b5 = F.relu(self.conv3m(x))
        b5 = F.max_pool2d(b5, kernel_size=5)
        b5 = b5.view(1,4800)
        b5 = F.relu(self.fc1m(b5))
        b5 = dropconnect(b5,0.5)
        b5 = F.relu(self.fc2m(b5))
        b5 = dropconnect(b5,0.5)
        b5 = self.fc3m(b5)
        
        b6 = F.relu(self.conv3o(x))
        b6 = F.max_pool2d(b6, kernel_size=5)
        b6 = b6.view(1,4800)
        b6 = self.others(b6)
        
        outputs = [b1,b2,b3,b4,b5,b6] 

        return torch.cat(outputs,1)
        
# ==============================================================
# ==============================================================


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # relu - 3*3 max pooling - 5*5 normalization
        self.conv1 = nn.Conv2d(3, 75, kernel_size = 7)
        
        # relu - 3*3 pooling - 5*5 normalization
        self.conv2 = nn.Conv2d(75, 200, kernel_size = 5)

        #         nn.BatchNorm2d(5*5)
        self.incept = InceptionA(in_channels = 200)
        self.fcaux = nn.Linear(40,40)
    
    def forward(self, x):
#         print(x.size(0))
        in_size = x.size(0)
        x = F.max_pool2d(F.relu(self.conv1(x)),(3,3))
        x = F.max_pool2d(F.relu(self.conv2(x)),(3,3))
        x = self.incept(x)
        x = self.fcaux(x)
#         print(x.shape())
        return x

# ========================================================

# Train the data

# ========================================================



file = open('output21.txt', 'w')
# sys.stdout = file
model = Net()
# model.cuda

# --- weight initialization --------
w1 = torch.Tensor(3, 5)
model.apply(init_weights)

def init_weights(m):
    if type(m) == nn.Conv2d :
        nn.init.xavier_uniform(m.weight, gain=np.sqrt(2))
        nn.init.kaiming_normal(w1, mode='fan_out')
#         init.constant(m.bias,0.0)

model = model.double()
print(model)
# file.write(model)
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.5)
criterion = nn.MSELoss()


def train(epoch):
    model.train()
    batch_idx = 0

    for (data, target) in zip(X_train,y_train):
        batch_idx = batch_idx+1
        target = torch.from_numpy(np.array(target,dtype='double')).double()
        data = torch.from_numpy(np.array(data,dtype='double')).double()
        data, target = Variable((data),requires_grad=True), Variable((target),requires_grad=True)
#         if args.cuda:
#             data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        data = data.view(1,3,img_size,img_size)
        output = model(data)
        i = 0
        loss_seq = []
        output = output.view(40,-1)
        out = output.data.numpy()
        tar = target.data.numpy()
        for o in out:
            if(i == 17):
                currloss = min(abs((o-tar[i+1])*(o-tar[i+1])),abs((o-tar[i])*(o-tar[i])))
                i = i+2
            else:
                currloss = abs((o-tar[i])*(o-tar[i]))
                i = i+1
            loss_seq.append(currloss)
      
        loss = sum(loss_seq)/40
        loss = torch.from_numpy(loss)
        output = torch.from_numpy(out)
        target = torch.from_numpy(tar)
        loss = Variable(loss,requires_grad=True).double()
        output = Variable(output,requires_grad=True).double()
        target = Variable(target,requires_grad=True).double()
        loss.backward()
        optimizer.step()
        if(batch_idx % 100 == 0):
#             file.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(y_train),
#                 100. * batch_idx / len(y_train), loss.data[0]))
            
            print('  Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}   '.format(
                epoch, batch_idx * len(data), len(y_train),
                100. * batch_idx / len(y_train), loss.data[0]),'\n')
            
            
def mse_loss(input, target):
    return torch.sum((input - target)**2) / input.data.nelement()
 
# ========================================================

# This is where you need your fingers crossed !! :| 

# ========================================================


def test():
    model.eval()
    batch_idx = 0
    test_loss = 0
    correct = 0
    
    for data, target in zip(X_test,y_test):
        target = torch.from_numpy(np.array(target,dtype='double')).double()
        data = torch.from_numpy(np.array(data,dtype='double')).double()
        data, target = Variable(data, volatile=True), Variable(target,requires_grad=True)
        
        data = data.view(1,3,img_size,img_size)
        loss_seq = []
        output = model(data)
        
        output = output.view(40,-1)
        out = output.data.numpy()
        tar = target.data.numpy() 

        mini = min(out)
        maxi = max(out)
        out = (out - mini)/float(maxi-mini)
##
        
        i = 0
        temp = 0       
        # sum up batch loss
        corr = 0
        for o in out:
            if(i == 17):
                currloss = min(abs((o-tar[i+1])*(o-tar[i+1])),abs((o-tar[i])*(o-tar[i])))
                if((o >= 0.5 and tar[i] >= 0.5) or (o <= 0.5 and tar[i] <= 0.5) or (o >= 0.5 and tar[i+1] >= 0.5) or (o <= 0.5 and tar[i+1] <= 0.5)):
                    correct = correct + 1
                i = i+2
      
            else:
                currloss = abs((o-tar[i])*(o-tar[i]))
                if((o >= 0.5 and tar[i] >= 0.5) or (o <= 0.5 and tar[i] <= 0.5)):
                    correct = correct + 1
                i = i+1
                
            loss_seq.append(currloss)
        test_loss += sum(loss_seq)/40
#         correct = correct + (corr/40)
        output = torch.from_numpy(out)
        target = torch.from_numpy(tar)
        output = Variable(output,requires_grad=True).double()
        target = Variable(target,requires_grad=True).double()
        
    test_loss /= len(y_test)
    print(len(y_test))
    print(test_loss)
    a = ("Test set: Average loss:",str(test_loss),"Accuracy: ",str(correct),"/",str(len(y_test))," = ",str((100*correct)/(len(y_test)*40)))
    aa = ''.join(a)
    print("Test set: Average loss:",test_loss,"Accuracy: ",correct,"/",len(y_test)," = ",((100*correct)/(len(y_test)*40)))
    file.write(aa)
#     file.write(("Test set: Average loss:",test_loss,"Accuracy: ",correct,"/",len(y_test)," = ",((100*correct)/(len(y_test)*40))))
#     print('\n Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%) \n'.format(test_loss, correct, str(len(y_test)), (100*correct)/(len(y_test)*40)))
#     file.write('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, str(len(y_test)), (100*correct)/(len(y_test)*40)))

for epoch in range(1, 5):
    train(epoch)
    test()
file.close()

Net(
  (conv1): Conv2d (3, 75, kernel_size=(7, 7), stride=(1, 1))
  (conv2): Conv2d (75, 200, kernel_size=(5, 5), stride=(1, 1))
  (incept): InceptionA(
    (conv3g): Conv2d (200, 300, kernel_size=(3, 3), stride=(1, 1))
    (fc1g): Linear(in_features=4800, out_features=512)
    (fc2g): Linear(in_features=512, out_features=512)
    (fc3g): Linear(in_features=512, out_features=1)
    (conv3e): Conv2d (200, 300, kernel_size=(3, 3), stride=(1, 1))
    (fc1e): Linear(in_features=4800, out_features=512)
    (fc2e): Linear(in_features=512, out_features=512)
    (fc3e): Linear(in_features=512, out_features=5)
    (conv3n): Conv2d (200, 300, kernel_size=(3, 3), stride=(1, 1))
    (fc1n): Linear(in_features=4800, out_features=512)
    (fc2n): Linear(in_features=512, out_features=512)
    (fc3n): Linear(in_features=512, out_features=2)
    (conv3f): Conv2d (200, 300, kernel_size=(3, 3), stride=(1, 1))
    (fc1f): Linear(in_features=4800, out_features=512)
    (fc2f): Linear(in_features=512, out_f

NameError: ignored